In [115]:
import pandas as pd
from pymongo import MongoClient
import requests
import json

In [116]:
# create a MongoDB client instance
client = MongoClient('mongodb://localhost:27017')

# get a reference to the database
db = client.mydatabase

# get a reference to the collection
collection = db.closing_prices_latest

# define your Alpha Vantage API key and endpoint URL
api_key = "HMU3C6GSU9RRXY5Z"
url = "https://www.alphavantage.co/query"

# define the symbol and data type to retrieve
symbol = ["AAPL","PAYTM.BSE"]
data_type = "TIME_SERIES_DAILY_ADJUSTED"

# define the API parameters
params = {
    "function": data_type,
    "symbol": symbol[1],
    "apikey": api_key
}

In [117]:
# make a request to the Alpha Vantage API
response = requests.get(url, params=params)

# convert the response data to a Python dictionary
data = json.loads(response.text)

In [118]:
# insert each day's closing price as a separate document
for date, values in data["Time Series (Daily)"].items():
    document = {
        "symbol": symbol[1],
        "timestamp": pd.to_datetime(date),
        "close": float(values["4. close"])
    }
    collection.insert_one(document)

In [119]:
# make a request to the Alpha Vantage API
response = requests.get(url, params=params)

# convert the response data to a Python dictionary
data = json.loads(response.text)

In [120]:
# get the latest timestamp in the collection, if any
latest_timestamp = collection.find_one(sort=[("timestamp", -1)])["timestamp"] if collection.count_documents({}) > 0 else None

In [121]:
# append new data to the collection
for date, values in data["Time Series (Daily)"].items():
    timestamp = pd.to_datetime(date)
    if latest_timestamp is None or timestamp > latest_timestamp:
        document = {
            "symbol": symbol[1],
            "timestamp": timestamp,
            "close": float(values["4. close"])
        }
        collection.update_one({"timestamp": timestamp}, {"$set": document}, upsert=True)

In [122]:
# query = "{ qty: { $gt: 4 } }"  # define your query here if necessary

results = collection.find( { "symbol": symbol[1] ,"close" :{ "$gt": 150} } )

df = pd.DataFrame(list(results))

print(df)

                         _id     symbol  timestamp   close
0   640b6070c95cf5831a629ce0  PAYTM.BSE 2023-03-09  606.25
1   640b6070c95cf5831a629ce1  PAYTM.BSE 2023-03-08  620.40
2   640b6070c95cf5831a629ce2  PAYTM.BSE 2023-03-06  625.30
3   640b6070c95cf5831a629ce3  PAYTM.BSE 2023-03-03  628.05
4   640b6070c95cf5831a629ce4  PAYTM.BSE 2023-03-02  604.45
..                       ...        ...        ...     ...
95  640b6070c95cf5831a629d3f  PAYTM.BSE 2022-10-21  629.40
96  640b6070c95cf5831a629d40  PAYTM.BSE 2022-10-20  632.85
97  640b6070c95cf5831a629d41  PAYTM.BSE 2022-10-19  652.40
98  640b6070c95cf5831a629d42  PAYTM.BSE 2022-10-18  662.90
99  640b6070c95cf5831a629d43  PAYTM.BSE 2022-10-17  672.70

[100 rows x 4 columns]
